In [1]:
# libraries that will be used for the project
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import re
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

ModuleNotFoundError: No module named 'matplotlib'